# 전체 시스템 흐름 테스트

이 노트북은 DebateAgent를 사용한 전체 토론 시스템의 흐름을 테스트합니다.

## 테스트 흐름
1. **환경 설정**: 필요한 라이브러리 import 및 설정
2. **데이터 준비**: 데이터셋 빌드 (선택적)
3. **Round 0**: 초기 의견 수집 (각 에이전트의 독립적 예측)
4. **Round 1-N**: 반박(Rebuttal) 및 의견 수정(Revise)
5. **최종 결과**: Ensemble 예측 및 결과 분석


In [1]:
# 필요한 라이브러리 import
import sys
import os
from datetime import datetime
from pathlib import Path

# 프로젝트 루트 경로 추가 (노트북 환경 대응)
notebook_dir = Path().resolve()  # 현재 노트북 디렉토리
project_root = notebook_dir.parent  # 프로젝트 루트 (notebooks의 상위 디렉토리)
sys.path.insert(0, str(project_root))

from agents.debate_agent import DebateAgent
from core.data_set import build_dataset

# 테스트 설정
TICKER = "NVDA"  # 테스트할 종목 티커
ROUNDS = 2  # 토론 라운드 수 (1-5 권장)

print(f"테스트 설정:")
print(f"  - 종목: {TICKER}")
print(f"  - 라운드 수: {ROUNDS}")
print(f"  - 시작 시간: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print(f"  - 프로젝트 루트: {project_root}")


테스트 설정:
  - 종목: NVDA
  - 라운드 수: 2
  - 시작 시간: 2025-11-16 14:55:04
  - 프로젝트 루트: /home/ubuntu/Projects/ml-ai/capstone


In [2]:
from agents.macro_agent import MacroAgent
from agents.sentimental_agent import SentimentalAgent
from agents.technical_agent import TechnicalAgent
from agents.debate_agent import DebateAgent



In [3]:
debate = DebateAgent(rounds=ROUNDS, ticker=TICKER)

In [4]:
debate.run()


[14:55:04] [1/4] 데이터셋 생성


[TechnicalAgent] X_seq: (982, 55, 13), y_seq: (982, 1)
✅ NVDA TechnicalAgent dataset saved to CSV (982 samples, 13 features)
[MacroAgent] X_seq: (1196, 40, 13), y_seq: (1196, 1)
✅ NVDA MacroAgent dataset saved to CSV (1196 samples, 13 features)
[SentimentalAgent] X_seq: (1196, 40, 8), y_seq: (1196, 1)


[**********            20%                       ]  3 of 15 completed

✅ NVDA SentimentalAgent dataset saved to CSV (1196 samples, 8 features)
✅ NVDA TechnicalAgent dataset saved via technical_data_set


[*********************100%***********************]  15 of 15 completed


...

[*********************100%***********************]  101 of 101 completed


저장 완료: (1259, 101) rows
macro: 데이터셋 생성> NVDA
✅ NVDA MacroAgent dataset saved (macro_dataset 호출 via relative: .macro_classes.macro_funcs)
[SentimentalAgent] X_seq: (468, 14, 15), y_seq: (468, 1)
✅ NVDA None dataset saved to CSV (468 samples, 15 features)
[14:55:16] 완료

[14:55:16] [2/4] Round 0 - 초기 Opinion 수집

  [1/3] TechnicalAgent
  |  [OK] 기존 모델 사용
  |  [OK] searcher 실행
  |  [OK] predict 실행
  |  [OK] reviewer_draft 실행
  └─ 결과: next_close=191.97, confidence=0.997

  [2/3] MacroAgent
  |  [OK] 기존 모델 사용
  |  [OK] searcher 실행
[INFO] 모델 및 스케일러 로드 중...
model_path: models/NVDA_MacroAgent.pt
[INFO] input_dim 불일치 감지: 13 -> 169, 레이어 재생성
[OK] 모델 및 스케일러 로드 완료
[INFO] MacroSentimentAgent 데이터 수집 중...
[INFO] Collecting macro features (15 tickers)...
[MacroSentimentAgent] Macro data: (43, 91)
[MacroSentimentAgent] Stock data: (43, 9)
[MacroSentimentAgent] Data shape: (43, 99)
[MacroSentimentAgent] Feature engineering complete. Final shape: (43, 170)
[OK] 매크로 데이터 수집 완료: (43, 171)
[INFO] 피처 정리 및 스케일링 중

2025-11-16 14:56:16.874697: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


[FinBERT] 7d_mean=-0.143 7d_cnt=100
[warn] Non-numeric features dropped in load_dataset(): ['date']
[warn] Non-numeric features dropped in load_dataset(): ['date']
[FinBERT] 캐시 탐색: /home/ubuntu/Projects/ml-ai/capstone/data/raw/news/NVDA.US_2025-10-05_2025-11-14.json (exists=True)
[FinBERT] 캐시 파일 로드 성공: 100건
[FinBERT] 100건 뉴스 감성 분석 시작...
[FinBERT] 7d_mean=-0.143 7d_cnt=100
[14:57:29] 완료 (6개)

[14:57:29] [3-2] Revision 수행
[TechnicalAgent] fine-tuning 완료: loss=0.014272
[TechnicalAgent] revise 완료 → new_close=191.92, loss=0.014272244647145271
[INFO] 모델 및 스케일러 로드 중...
model_path: models/NVDA_MacroAgent.pt
[OK] 모델 및 스케일러 로드 완료
[INFO] MacroSentimentAgent 데이터 수집 중...
[INFO] Collecting macro features (15 tickers)...
[MacroSentimentAgent] Macro data: (43, 91)
[MacroSentimentAgent] Stock data: (43, 9)
[MacroSentimentAgent] Data shape: (43, 99)
[MacroSentimentAgent] Feature engineering complete. Final shape: (43, 170)
[OK] 매크로 데이터 수집 완료: (43, 171)
[INFO] 피처 정리 및 스케일링 중...
[Check] 입력 피처 수: 169 / 스케일

{'ticker': 'NVDA',
 'agents': {'TechnicalAgent_next_close': 191.87957145680406,
  'MacroAgent_next_close': 191.5309468156059,
  'SentimentalAgent_next_close': 192.08579750114777},
 'mean_next_close': 191.8321052578526,
 'median_next_close': 191.87957145680406,
 'currency': 'USD',
 'last_price': 190.17}

In [5]:
debate.opinions

{0: {'TechnicalAgent': Opinion(agent_id='TechnicalAgent', target=Target(next_close=191.97268769977055, uncertainty=0.0033974405378103256, confidence=0.9966197823490364), reason='1) ma_200 (0.3132×0.0216=0.0068): 200일 이동평균선은 장기적인 주가 흐름을 보여주며, 주가가 이 선 위에 있으면 상승 추세로 해석됩니다. 2) mom_10 (0.1249×0.0216=0.0027): 10일 모멘텀은 단기 가격 변화 속도를 나타내어 상승 모멘텀이 강함을 의미합니다. 3) bbp (0.0870×0.0216=0.0019): 볼린저 밴드 퍼센트는 주가가 밴드 내 위치를 나타내며, 상단에 가까울수록 강한 매수세를 시사합니다. 4) weekofyear_cos (0.0658×0.0216=0.0014): 연중 주기성을 반영하여 특정 시기에 주가가 상승하는 경향을 보여줍니다. 이들 상위 시점들은 2025-06-10~2025-08-27 기간에 집중되어 있어, 이 기간 동안 주가가 꾸준한 상승세와 거래 활발함을 보였음을 알 수 있습니다. 영향 점수에서 ma_200이 가장 높아 장기 상승 추세가 강력한 기반임을 나타내며, 다음으로 mom_10과 bbp가 단기 모멘텀과 매수세를 뒷받침해 상승 신호를 강화합니다. weekofyear_cos는 계절적 요인을 반영하지만 영향 점수가 상대적으로 낮아 보조적 역할에 그칩니다. 반면, 과열 신호나 되돌림 신호는 영향 점수가 낮아 예측에 큰 제약을 주지 않습니다. 따라서 장기 추세와 단기 모멘텀, 매수세가 조화를 이루어 다음날 종가 상승 예측이 사용자 입장에서 신뢰할 만하며 안정적인 상승 방향성을 기대할 수 있습니다.'),
  'MacroAgent': Opinion(agent_id='MacroAgent', target=Target(next_close=190.51133851388568, un

In [5]:
build_dataset(TICKER)

[*********************100%***********************]  1 of 1 completed


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


[TechnicalAgent] X_seq: (982, 55, 13), y_seq: (982, 1)
✅ NVDA TechnicalAgent dataset saved to CSV (982 samples, 13 features)
[MacroAgent] X_seq: (1222, 14, 13), y_seq: (1222, 1)
✅ NVDA MacroAgent dataset saved to CSV (1222 samples, 13 features)
[SentimentalAgent] X_seq: (1196, 40, 8), y_seq: (1196, 1)


[**********            20%                       ]  3 of 15 completed

✅ NVDA SentimentalAgent dataset saved to CSV (1196 samples, 8 features)
✅ NVDA TechnicalAgent dataset saved via technical_data_set
[TRACE B] macro_dataset() start for NVDA


...

[MacroSentimentAgent] Data shape: (1304, 90), Columns: 90
[TRACE A] add_features() for self.data:            Open_CL=F  Open_DX-Y.NYB  Open_EURUSD=X    Open_GC=F  Open_HG=F  \
Date                                                                          
2020-01-01        NaN            NaN       1.122083          NaN        NaN   
2020-01-02  61.599998      96.480003       1.121894  1518.099976     2.8165   
2020-01-03  61.180000      96.790001       1.117081  1530.099976     2.7935   
2020-01-06  63.709999      96.900002       1.116246  1580.000000     2.7780   
2020-01-07  62.910000      96.650002       1.119583  1558.300049     2.8010   
...               ...            ...            ...          ...        ...   
2024-12-24  69.559998     108.160004       1.040583  2613.000000     4.0525   
2024-12-25        NaN            NaN       1.040258          NaN        NaN   
2024-12-26  70.199997     108.169998       1.039955  2628.500000     4.0730   
2024-12-27  69.680000     108.0800

[*                      3%                       ]  3 of 101 completed

[MacroSentimentAgent] Saved data/processed/NVDA_MacroAgent.csv


[*********************100%***********************]  101 of 101 completed


저장 완료: (1259, 101) rows
macro: 데이터셋 생성> NVDA
✅ NVDA MacroAgent dataset saved (macro_dataset 호출 via relative: .macro_classes.macro_funcs)
[SentimentalAgent] X_seq: (468, 14, 15), y_seq: (468, 1)
✅ NVDA None dataset saved to CSV (468 samples, 15 features)


In [6]:
debate.get_opinion(0, TICKER)

[TechnicalAgent] pretrain 실행 (모델/스케일러 없음)
[12:52:09] Pretraining TechnicalAgent
  Epoch 005 | Loss: 0.392349
  Epoch 010 | Loss: 0.385570
  Epoch 015 | Loss: 0.383381
  Epoch 020 | Loss: 0.383638
  Epoch 025 | Loss: 0.396127
  Epoch 030 | Loss: 0.383934
  Epoch 035 | Loss: 0.394238
  Epoch 040 | Loss: 0.393365


[*********************100%***********************]  1 of 1 completed

  Epoch 045 | Loss: 0.385185
 TechnicalAgent 모델 학습 및 저장 완료: models/NVDA_TechnicalAgent.pt
[TechnicalAgent] searcher 실행
⚙️ NVDA TechnicalAgent rebuild requested. Building dataset...



[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


[TechnicalAgent] X_seq: (982, 55, 13), y_seq: (982, 1)
✅ NVDA TechnicalAgent dataset saved to CSV (982 samples, 13 features)
[MacroAgent] X_seq: (1222, 14, 13), y_seq: (1222, 1)
✅ NVDA MacroAgent dataset saved to CSV (1222 samples, 13 features)
[SentimentalAgent] X_seq: (1196, 40, 8), y_seq: (1196, 1)
✅ NVDA SentimentalAgent dataset saved to CSV (1196 samples, 8 features)
■ TechnicalAgent StockData 생성 완료 (NVDA, USD)
[TechnicalAgent] predict 실행
[TechnicalAgent] reviewer_draft 실행
■ TechnicalAgent StockData 생성 완료 (NVDA, USD)


...

[*                      3%                       ]  3 of 101 completed

[MacroSentimentAgent] Saved data/processed/NVDA_MacroAgent.csv


[*********************100%***********************]  101 of 101 completed


저장 완료: (1259, 101) rows
macro: 데이터셋 생성> NVDA
✅ NVDA MacroAgent dataset saved (macro_dataset 호출 via relative: .macro_classes.macro_funcs)
[SentimentalAgent] X_seq: (442, 40, 8), y_seq: (442, 1)
✅ NVDA SentimentalAgent dataset saved to CSV (442 samples, 8 features)
■ SentimentalAgent StockData 생성 완료 (NVDA, USD)
[SentimentalAgent] predict 실행
[SentimentalAgent] reviewer_draft 실행
  - SentimentalAgent: next_close=187.1166
 Round 0 의견 수집 완료 (3 agents)


{'TechnicalAgent': Opinion(agent_id='TechnicalAgent', target=Target(next_close=189.99532882954946, uncertainty=0.004526279866695404, confidence=0.9955042361035691), reason='1) ma_200 (0.2445×0.0209=0.0051): 200일 이동평균선은 장기적인 가격 추세를 보여주며, 주가가 이 선 위에 있으면 상승 추세로 해석됩니다. 2) obv (0.1017×0.0209=0.0021): 거래량 기반 지표로, 매수와 매도 압력을 파악하는 데 도움을 줍니다. 3) macd (0.0949×0.0209=0.0020): 단기와 장기 이동평균선의 차이를 이용해 추세 전환 신호를 포착합니다. 4) mom_10 (0.0844×0.0209=0.0018): 10일 모멘텀 지표로 최근 가격 변화 속도를 나타냅니다. 상위 시점들은 2025-08-12부터 2025-10-28까지 인접하며, 이 기간 동안 주가는 전반적으로 안정적인 상승 흐름을 보였습니다. ma_200의 영향 점수가 가장 커 장기 상승 추세가 강하게 작용했음을 알 수 있고, obv와 macd도 긍정적인 거래량과 추세 신호를 뒷받침합니다. mom_10은 상대적으로 작은 영향 점수로 단기 변동성은 있으나 전체 흐름에 큰 영향을 미치지 않았습니다. 일부 되돌림 신호가 있었으나 영향 점수(0.0018 이하)가 낮아 전체 상승 추세에 큰 제약이 되지 않았습니다. 따라서 장기 추세와 거래량 지표가 안정적인 상승을 지지해 다음날 종가 예측이 신뢰할 만하며, 가격 변동성도 낮아 예측의 안정성이 높다고 판단됩니다.'),
 'MacroAgent': Opinion(agent_id='MacroAgent', target=Target(next_close=187.79147523024162, uncertainty=0.03722068667411804, confidence=0.9647437119342431), r

In [7]:
debate.get_rebuttal(1)
debate.get_revise(1)

[FinBERT] 캐시 탐색: /home/ubuntu/Projects/ml-ai/capstone/data/raw/news/NVDA.US_2025-10-05_2025-11-14.json (exists=False)
[FinBERT] 뉴스 캐시 없음: /home/ubuntu/Projects/ml-ai/capstone/data/raw/news/NVDA.US_2025-10-05_2025-11-14.json (글로벌 매치도 없음)
[FinBERT] 캐시 탐색: /home/ubuntu/Projects/ml-ai/capstone/data/raw/news/NVDA.US_2025-10-05_2025-11-14.json (exists=False)
[FinBERT] 뉴스 캐시 없음: /home/ubuntu/Projects/ml-ai/capstone/data/raw/news/NVDA.US_2025-10-05_2025-11-14.json (글로벌 매치도 없음)
■ TechnicalAgent StockData 생성 완료 (NVDA, USD)
[TechnicalAgent] fine-tuning 완료: loss=0.057061
■ TechnicalAgent StockData 생성 완료 (NVDA, USD)


/home/ubuntu/Projects/ml-ai/capstone/capstone/lib/python3.10/site-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
[*******************   40%                       ]  6 of 15 completed

[TechnicalAgent] revise 완료 → new_close=189.94, loss=0.05706050619482994
[INFO] 모델 및 스케일러 로드 중...
model_path: models/NVDA_MacroAgent.pt
[OK] 모델 및 스케일러 로드 완료
[INFO] MacroSentimentAgent 데이터 수집 중...
1️⃣ Collecting macro features (15 tickers)...


[*********************100%***********************]  15 of 15 completed
[*********************100%***********************]  1 of 1 completed


[MacroSentimentAgent] Macro data: (43, 91)
   ↳ Downloading NVDA ...
[MacroSentimentAgent] Stock data: (43, 9)
[MacroSentimentAgent] Data shape: (43, 99)
[MacroSentimentAgent] Feature engineering complete. Final shape: (43, 170)
[OK] 매크로 데이터 수집 완료: (43, 171)
[INFO] 피처 정리 및 스케일링 중...
[Check] 입력 피처 수: 169 / 스케일러 기준 피처 수: 169
[OK] 스케일링 및 시퀀스 변환 완료
■ MacroAgent StockData 생성 완료 (NVDA, USD)
[MacroAgent] fine-tuning 완료: loss=0.004285
[MacroAgent] revise 완료 → new_close=188.73, loss=0.004285294096916914
■ SentimentalAgent StockData 생성 완료 (NVDA, USD)
[FinBERT] 캐시 탐색: /home/ubuntu/Projects/ml-ai/capstone/data/raw/news/NVDA.US_2025-10-05_2025-11-14.json (exists=False)
[FinBERT] 뉴스 캐시 없음: /home/ubuntu/Projects/ml-ai/capstone/data/raw/news/NVDA.US_2025-10-05_2025-11-14.json (글로벌 매치도 없음)
[SentimentalAgent] revise 완료 → new_close=187.88, loss=10.45433235168457
 Round 1 revise 완료 및 opinions 갱신 (3 agents)


{'TechnicalAgent': Opinion(agent_id='TechnicalAgent', target=Target(next_close=189.9414604733962, uncertainty=0.0050717624835669994, confidence=0.9949665196790966), reason='기존 의견에서는 NVDA 주가가 200일 이동평균(ma_200) 위에 위치하며 장기 상승 추세가 강하게 작용하고, OBV와 MACD 지표도 긍정적인 거래량과 추세 신호를 뒷받침해 다음날 종가가 상승할 것으로 예측하셨습니다. 즉, 현재가 대비 상승 방향에 무게를 두셨습니다. 동료 에이전트들의 의견을 보면 MacroAgent는 거시경제 환경의 금리 상승과 정책 긴축 기조, 원자재 변동성 확대를 근거로 소폭 하락을 예상하며 보수적인 하락 전망을 제시했고, SentimentalAgent는 단기 긍정 뉴스와 감성 개선을 근거로 상승을 보았으나 뉴스 데이터 부재로 신뢰성에 의문을 제기받았습니다. 반박 요약으로는 MacroAgent가 거시경제 긴축과 원자재 변동성 확대가 단기 리스크임을 강조하며 기존 상승 전망이 거시경제 흐름과 상충한다고 지적했고, SentimentalAgent는 감성 데이터 부재로 단기 예측 신뢰성에 한계가 있다고 했습니다. 저는 거시경제 리스크를 완전히 배제하지 않고 일정 부분 수용하되, 기술적 지표의 강한 장기 상승 신호와 거래량 기반 OBV, MACD의 긍정적 흐름을 중점적으로 반영했습니다. 최근 14일간 가격·추세 지표에서는 200일 이동평균이 꾸준히 상승하며 주가를 지지하고, MACD도 상승세를 유지해 단기와 중기 모두 상승 추세를 나타냅니다. 모멘텀 지표인 mom_10은 일부 단기 조정 신호를 보이나 영향력이 작아 전체 추세에 큰 제약이 되지 않으며, 거래량과 변동성 지표는 다소 변동성이 있으나 급격한 거래량 급증이나 급감은 없어 안정적인 흐름으로 판단됩니다. 따라서 이번 수정에서는 기존의 상승 전망에 다소 보수적인 시각을 더해, 상승 쪽

In [8]:
debate.get_rebuttal(2)
debate.get_revise(2)

[FinBERT] 캐시 탐색: /home/ubuntu/Projects/ml-ai/capstone/data/raw/news/NVDA.US_2025-10-05_2025-11-14.json (exists=False)
[FinBERT] 뉴스 캐시 없음: /home/ubuntu/Projects/ml-ai/capstone/data/raw/news/NVDA.US_2025-10-05_2025-11-14.json (글로벌 매치도 없음)
[FinBERT] 캐시 탐색: /home/ubuntu/Projects/ml-ai/capstone/data/raw/news/NVDA.US_2025-10-05_2025-11-14.json (exists=False)
[FinBERT] 뉴스 캐시 없음: /home/ubuntu/Projects/ml-ai/capstone/data/raw/news/NVDA.US_2025-10-05_2025-11-14.json (글로벌 매치도 없음)
■ TechnicalAgent StockData 생성 완료 (NVDA, USD)
[TechnicalAgent] fine-tuning 완료: loss=0.018211
■ TechnicalAgent StockData 생성 완료 (NVDA, USD)


/home/ubuntu/Projects/ml-ai/capstone/capstone/lib/python3.10/site-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
[*******************   40%                       ]  6 of 15 completed

[TechnicalAgent] revise 완료 → new_close=189.86, loss=0.018210798501968384
[INFO] 모델 및 스케일러 로드 중...
model_path: models/NVDA_MacroAgent.pt
[OK] 모델 및 스케일러 로드 완료
[INFO] MacroSentimentAgent 데이터 수집 중...
1️⃣ Collecting macro features (15 tickers)...


[*********************100%***********************]  15 of 15 completed
[*********************100%***********************]  1 of 1 completed


[MacroSentimentAgent] Macro data: (43, 91)
   ↳ Downloading NVDA ...
[MacroSentimentAgent] Stock data: (43, 9)
[MacroSentimentAgent] Data shape: (43, 99)
[MacroSentimentAgent] Feature engineering complete. Final shape: (43, 170)
[OK] 매크로 데이터 수집 완료: (43, 171)
[INFO] 피처 정리 및 스케일링 중...
[Check] 입력 피처 수: 169 / 스케일러 기준 피처 수: 169
[OK] 스케일링 및 시퀀스 변환 완료
■ MacroAgent StockData 생성 완료 (NVDA, USD)
[MacroAgent] fine-tuning 완료: loss=0.000802
[MacroAgent] revise 완료 → new_close=189.23, loss=0.0008022705442272127
■ SentimentalAgent StockData 생성 완료 (NVDA, USD)
[FinBERT] 캐시 탐색: /home/ubuntu/Projects/ml-ai/capstone/data/raw/news/NVDA.US_2025-10-05_2025-11-14.json (exists=False)
[FinBERT] 뉴스 캐시 없음: /home/ubuntu/Projects/ml-ai/capstone/data/raw/news/NVDA.US_2025-10-05_2025-11-14.json (글로벌 매치도 없음)
[SentimentalAgent] revise 완료 → new_close=188.44, loss=6.61091947555542
 Round 2 revise 완료 및 opinions 갱신 (3 agents)


{'TechnicalAgent': Opinion(agent_id='TechnicalAgent', target=Target(next_close=189.8573822756335, uncertainty=0.004739286378026009, confidence=0.9952941583316629), reason='기존 의견에서는 NVDA 주가가 200일 이동평균 위에 위치하며 장기 상승 추세가 강하게 작용하고, OBV와 MACD 지표도 긍정적인 거래량과 추세 신호를 뒷받침해 다음날 종가가 상승할 것으로 예측하셨습니다. 즉, 현재가 대비 상승 방향에 무게를 두셨습니다. 동료 에이전트들의 의견을 보면 MacroAgent는 금리 상승과 유동성 축소, 원자재 변동성 확대라는 거시경제 환경을 근거로 보수적인 하락 전망을 제시했고, SentimentalAgent는 뉴스 부재와 감성 지표 부재를 이유로 중립적이거나 소폭 하락을 예상하며 보수적 입장을 보였습니다. 반박 요약에서는 거시경제 긴축과 원자재 변동성 확대가 단기 리스크임을 강조하며 기존 상승 전망이 거시경제 흐름과 상충한다는 지적이 있었고, 감성 데이터 부재로 단기 예측 신뢰성에 한계가 있다는 점도 지적되었습니다. 저는 이 중 거시경제 리스크를 완전히 배제하지 않고 일정 부분 수용하되, 기술적 지표의 강한 장기 상승 신호와 거래량 기반 OBV, MACD의 긍정적 흐름을 중점적으로 반영하였습니다. 최근 14일간 가격·추세 지표에서는 200일 이동평균이 꾸준히 상승하며 주가를 지지하고, MACD도 상승세를 유지해 단기와 중기 모두 상승 추세를 나타냅니다. 모멘텀 지표인 mom_10은 일부 단기 조정 신호를 보이나 전체 추세에 큰 제약이 되지 않으며, 거래량과 변동성 지표는 다소 변동성이 있으나 급격한 거래량 급증이나 급감은 없어 안정적인 흐름으로 판단됩니다. 따라서 이번 수정에서는 기존의 상승 전망에 다소 보수적인 시각을 더해, 상승 쪽에 무게를 두되 거시경제 리스크로 인한 단기 변동성 가능성도 함께 고려하는 방향으로 조정하

In [7]:
macro = MacroAgent(ticker=TICKER)
technical = TechnicalAgent(ticker=TICKER)
sentimental = SentimentalAgent(ticker=TICKER)

In [13]:
data = technical.searcher()

■ TechnicalAgent StockData 생성 완료 (NVDA, USD)


In [11]:
technical.pretrain()

[11:48:57] Pretraining TechnicalAgent
  Epoch 005 | Loss: 0.381739
  Epoch 010 | Loss: 0.382532
  Epoch 015 | Loss: 0.381650
  Epoch 020 | Loss: 0.386132
  Epoch 025 | Loss: 0.386860
  Epoch 030 | Loss: 0.383978
  Epoch 035 | Loss: 0.386244
  Epoch 040 | Loss: 0.383551
  Epoch 045 | Loss: 0.395979
 TechnicalAgent 모델 학습 및 저장 완료: models/NVDA_TechnicalAgent.pt


In [14]:
technical.predict(data)

Target(next_close=190.63277158716582, uncertainty=0.0047906432300806046, confidence=0.9952435275386796)

In [15]:
technical.reviewer_draft()

■ TechnicalAgent StockData 생성 완료 (NVDA, USD)
■ TechnicalAgent StockData 생성 완료 (NVDA, USD)


TypeError: unsupported operand type(s) for +: 'int' and 'list'

## 1. DebateAgent 초기화

세 가지 에이전트를 포함한 DebateAgent를 초기화합니다:
- **TechnicalAgent**: 기술적 분석가
- **MacroSentiAgent**: 거시경제 분석가  
- **SentimentalAgent**: 센티멘탈 분석가


In [4]:
# DebateAgent 초기화
print("=" * 60)
print("■ DebateAgent 초기화 중...")
print("=" * 60)

debate_agent = DebateAgent(rounds=ROUNDS, ticker=TICKER)

print(f"\n✅ DebateAgent 초기화 완료")
print(f"   - 에이전트 수: {len(debate_agent.agents)}")
print(f"   - 에이전트 목록: {list(debate_agent.agents.keys())}")
print(f"   - 토론 라운드: {debate_agent.rounds}")


■ DebateAgent 초기화 중...

✅ DebateAgent 초기화 완료
   - 에이전트 수: 3
   - 에이전트 목록: ['TechnicalAgent', 'MacroSentiAgent', 'SentimentalAgent']
   - 토론 라운드: 2


## 2. 데이터셋 준비 (선택적)

데이터셋이 이미 존재하는 경우 이 단계를 건너뛸 수 있습니다.


In [6]:
# 데이터셋 빌드 (필요한 경우에만 실행)
# 주석을 해제하여 실행하세요
print("=" * 60)
print("■ 데이터셋 빌드 중...")
print("=" * 60)
build_dataset(TICKER)
print("✅ 데이터셋 빌드 완료\n")


■ 데이터셋 빌드 중...


[*********************100%***********************]  1 of 1 completed


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


[TechnicalAgent] X_seq: (982, 55, 13), y_seq: (982, 1)
✅ NVDA TechnicalAgent dataset saved to CSV (982 samples, 13 features)
[MacroSentiAgent] X_seq: (1222, 14, 13), y_seq: (1222, 1)
✅ NVDA MacroSentiAgent dataset saved to CSV (1222 samples, 13 features)
[SentimentalAgent] X_seq: (1196, 40, 8), y_seq: (1196, 1)


[**********            20%                       ]  3 of 15 completed

...

[*********************100%***********************]  101 of 101 completed


저장 완료: (1259, 101) rows
macro: 데이터셋 생성> NVDA
[INFO] Removed all constant Volume columns matching patterns: ['Volume_^FVX', 'Volume_^IRX', 'Volume_^TNX', 'Volume_^VIX', 'Volume_DX-Y.NYB', 'Volume_EURUSD=X', 'Volume_USDJPY=X']
[INFO] Remaining Volume columns: ['Volume_CL=F', 'Volume_GC=F', 'Volume_HG=F', 'Volume_QQQ', 'Volume_SPY', 'Volume_^DJI', 'Volume_^GSPC', 'Volume_^IXIC', 'Volume_CL=F_ret', 'Volume_GC=F_ret', 'Volume_HG=F_ret', 'Volume_QQQ_ret', 'Volume_SPY_ret', 'Volume_^DJI_ret', 'Volume_^GSPC_ret', 'Volume_^IXIC_ret']
[INFO] 병합 후 데이터 shape: (1257, 271)


2025-11-16 10:19:22.576587: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


Epoch 1/60
55/55 ━━━━━━━━━━━━━━━━━━━━ 8s 55ms/step - loss: 0.1462 - val_loss: 0.0897
Epoch 2/60
55/55 ━━━━━━━━━━━━━━━━━━━━ 3s 47ms/step - loss: 0.1333 - val_loss: 0.0908
Epoch 3/60
55/55 ━━━━━━━━━━━━━━━━━━━━ 3s 48ms/step - loss: 0.1311 - val_loss: 0.1373
Epoch 4/60
55/55 ━━━━━━━━━━━━━━━━━━━━ 3s 51ms/step - loss: 0.1292 - val_loss: 0.0837
Epoch 5/60
55/55 ━━━━━━━━━━━━━━━━━━━━ 3s 48ms/step - loss: 0.1235 - val_loss: 0.0854
Epoch 6/60
55/55 ━━━━━━━━━━━━━━━━━━━━ 3s 53ms/step - loss: 0.1264 - val_loss: 0.1048
Epoch 7/60
55/55 ━━━━━━━━━━━━━━━━━━━━ 3s 49ms/step - loss: 0.1242 - val_loss: 0.0826
Epoch 8/60
55/55 ━━━━━━━━━━━━━━━━━━━━ 3s 49ms/step - loss: 0.1221 - val_loss: 0.0822
Epoch 9/60
55/55 ━━━━━━━━━━━━━━━━━━━━ 3s 47ms/step - loss: 0.1249 - val_loss: 0.0814
Epoch 10/60
55/55 ━━━━━━━━━━━━━━━━━━━━ 4s 65ms/step - loss: 0.1210 - val_loss: 0.0818
Epoch 11/60
55/55 ━━━━━━━━━━━━━━━━━━━━ 3s 48ms/step - loss: 0.1221 - val_loss: 0.0819
Epoch 12/60
55/55 ━━━━━━━━━━━━━━━━━━━━ 3s 50ms/step - loss: 0.1

FileNotFoundError: [Errno 2] No such file or directory: 'models/NVDA_MacroSentiAgent.keras'

## 3. Round 0: 초기 의견 수집

각 에이전트가 독립적으로 주식을 분석하고 예측합니다.


In [7]:
# Round 0: 초기 의견 수집
print("=" * 60)
print("■ Round 0: 초기 의견 수집 시작")
print("=" * 60)

opinions_0 = debate_agent.get_opinion(round=0, ticker=TICKER, rebuild=True, force_pretrain=False)

print("\n" + "=" * 60)
print("✅ Round 0 완료: 초기 의견 수집 결과")
print("=" * 60)

for agent_id, opinion in opinions_0.items():
    if opinion and opinion.target:
        print(f"\n[{agent_id}]")
        print(f"  예측 종가: ${opinion.target.next_close:.2f}")
        print(f"  불확실성(σ): {opinion.target.uncertainty:.6f}" if opinion.target.uncertainty else "  불확실성: N/A")
        print(f"  신뢰도(β): {opinion.target.confidence:.4f}" if opinion.target.confidence else "  신뢰도: N/A")
        print(f"  근거 요약: {opinion.reason[:100]}..." if len(opinion.reason) > 100 else f"  근거: {opinion.reason}")


[*********************100%***********************]  1 of 1 completed

■ Round 0: 초기 의견 수집 시작
[TechnicalAgent] searcher 실행
⚙️ NVDA TechnicalAgent rebuild requested. Building dataset...



[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


[TechnicalAgent] X_seq: (982, 55, 13), y_seq: (982, 1)
✅ NVDA TechnicalAgent dataset saved to CSV (982 samples, 13 features)
[MacroSentiAgent] X_seq: (1222, 14, 13), y_seq: (1222, 1)
✅ NVDA MacroSentiAgent dataset saved to CSV (1222 samples, 13 features)
[SentimentalAgent] X_seq: (1196, 40, 8), y_seq: (1196, 1)
✅ NVDA SentimentalAgent dataset saved to CSV (1196 samples, 8 features)
■ TechnicalAgent StockData 생성 완료 (NVDA, USD)
[TechnicalAgent] pretrain 실행
[10:28:38] Pretraining TechnicalAgent
  Epoch 005 | Loss: 0.375417
  Epoch 010 | Loss: 0.395027
  Epoch 015 | Loss: 0.393638
  Epoch 020 | Loss: 0.381625
  Epoch 025 | Loss: 0.388605
  Epoch 030 | Loss: 0.388451
  Epoch 035 | Loss: 0.398935
  Epoch 040 | Loss: 0.383987
  Epoch 045 | Loss: 0.389688
 TechnicalAgent 모델 학습 및 저장 완료: models/NVDA_TechnicalAgent.pt
[TechnicalAgent] predict 실행
[TechnicalAgent] reviewer_draft 실행
■ TechnicalAgent StockData 생성 완료 (NVDA, USD)
  - TechnicalAgent: next_close=190.5389
MacroSentiAgent의 데이터 로드.. macro_s

ValueError: File not found: filepath=models/NVDA_MacroSentiAgent.keras. Please ensure the file is an accessible `.keras` zip file.

## 4. Round 1-N: 반박 및 의견 수정

각 라운드마다:
1. **Rebuttal**: 다른 에이전트의 의견에 대한 반박/지지
2. **Revise**: 신뢰도 기반으로 자신의 의견 수정


In [ ]:
# Round 1-N: 반박 및 의견 수정
for round_num in range(1, ROUNDS + 1):
    print("\n" + "=" * 60)
    print(f"■ Round {round_num}: 반박 및 의견 수정")
    print("=" * 60)
    
    # 4-1. Rebuttal (반박)
    print(f"\n[Round {round_num}-1] Rebuttal 수행 중...")
    rebuttals = debate_agent.get_rebuttal(round=round_num)
    print(f"✅ Rebuttal 완료: {len(rebuttals)}개의 반박 생성")
    
    # 반박 내용 일부 출력
    for rebuttal in rebuttals[:2]:  # 처음 2개만 출력
        print(f"\n  [{rebuttal.from_agent_id} → {rebuttal.to_agent_id}]")
        print(f"    스탠스: {rebuttal.stance}")
        print(f"    메시지: {rebuttal.message[:150]}...")
    
    # 4-2. Revise (의견 수정)
    print(f"\n[Round {round_num}-2] Revise 수행 중...")
    revised_opinions = debate_agent.get_revise(round=round_num)
    print(f"✅ Revise 완료: {len(revised_opinions)}개 에이전트 의견 수정")
    
    # 수정된 의견 출력
    print(f"\n[Round {round_num}] 수정된 의견:")
    for agent_id, opinion in revised_opinions.items():
        if opinion and opinion.target:
            prev_opinion = debate_agent.opinions[round_num - 1][agent_id]
            prev_price = prev_opinion.target.next_close if prev_opinion and prev_opinion.target else None
            
            change = opinion.target.next_close - prev_price if prev_price else 0
            change_pct = (change / prev_price * 100) if prev_price else 0
            
            print(f"\n  [{agent_id}]")
            print(f"    이전 예측: ${prev_price:.2f}" if prev_price else "    이전 예측: N/A")
            print(f"    수정 예측: ${opinion.target.next_close:.2f}")
            print(f"    변화: ${change:+.2f} ({change_pct:+.2f}%)")
            print(f"    불확실성: {opinion.target.uncertainty:.6f}" if opinion.target.uncertainty else "    불확실성: N/A")
    
    print(f"\n✅ Round {round_num} 토론 완료")


## 5. 최종 결과: Ensemble 예측

모든 라운드가 완료된 후 최종 예측 결과를 확인합니다.


In [ ]:
# 최종 결과: Ensemble 예측
print("\n" + "=" * 60)
print("■ 최종 결과: Ensemble 예측")
print("=" * 60)

ensemble_result = debate_agent.get_ensemble()

print("\n📊 최종 예측 결과:")
print(f"  종목: {ensemble_result['ticker']}")
print(f"  통화: {ensemble_result['currency']}")
print(f"  현재가: ${ensemble_result['last_price']:.2f}" if ensemble_result['last_price'] else "  현재가: N/A")
print(f"\n  에이전트별 예측:")
for key, value in ensemble_result['agents'].items():
    agent_name = key.replace('_next_close', '')
    print(f"    {agent_name}: ${value:.2f}")

print(f"\n  앙상블 예측:")
print(f"    평균: ${ensemble_result['mean_next_close']:.2f}" if ensemble_result['mean_next_close'] else "    평균: N/A")
print(f"    중앙값: ${ensemble_result['median_next_close']:.2f}" if ensemble_result['median_next_close'] else "    중앙값: N/A")

# 현재가와 비교
if ensemble_result['last_price'] and ensemble_result['mean_next_close']:
    current = ensemble_result['last_price']
    predicted = ensemble_result['mean_next_close']
    change = predicted - current
    change_pct = (change / current) * 100
    print(f"\n  예상 수익률:")
    print(f"    절대 변화: ${change:+.2f}")
    print(f"    상대 변화: {change_pct:+.2f}%")


## 6. 라운드별 의견 변화 시각화

각 라운드별로 에이전트의 예측이 어떻게 변화했는지 확인합니다.


In [ ]:
# 라운드별 의견 변화 분석
import pandas as pd
import matplotlib.pyplot as plt

print("\n" + "=" * 60)
print("■ 라운드별 의견 변화 분석")
print("=" * 60)

# 라운드별 의견 데이터 수집
rounds_data = []
for round_num in sorted(debate_agent.opinions.keys()):
    opinions = debate_agent.opinions[round_num]
    for agent_id, opinion in opinions.items():
        if opinion and opinion.target:
            rounds_data.append({
                'Round': round_num,
                'Agent': agent_id,
                'Predicted_Price': opinion.target.next_close,
                'Uncertainty': opinion.target.uncertainty or 0,
                'Confidence': opinion.target.confidence or 0
            })

df_rounds = pd.DataFrame(rounds_data)

if not df_rounds.empty:
    # 라운드별 예측가 변화 시각화
    plt.figure(figsize=(12, 6))
    
    for agent_id in df_rounds['Agent'].unique():
        agent_data = df_rounds[df_rounds['Agent'] == agent_id].sort_values('Round')
        plt.plot(agent_data['Round'], agent_data['Predicted_Price'], 
                marker='o', label=agent_id, linewidth=2, markersize=8)
    
    plt.xlabel('Round', fontsize=12)
    plt.ylabel('Predicted Price ($)', fontsize=12)
    plt.title(f'Round별 예측가 변화 ({TICKER})', fontsize=14, fontweight='bold')
    plt.legend(loc='best', fontsize=10)
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()
    
    # 테이블로도 출력
    print("\n📋 라운드별 예측가 변화 테이블:")
    pivot_table = df_rounds.pivot_table(
        index='Agent', 
        columns='Round', 
        values='Predicted_Price',
        aggfunc='first'
    )
    print(pivot_table.round(2))
else:
    print("데이터가 없습니다.")


## 7. 간편 실행: run() 메서드 사용

위의 단계를 한 번에 실행하려면 `run()` 메서드를 사용할 수 있습니다.


In [ ]:
# 간편 실행 (주석 해제하여 사용)
# debate_agent_simple = DebateAgent(rounds=ROUNDS, ticker=TICKER)
# debate_agent_simple.run()
# print(debate_agent_simple.get_ensemble())


## 8. 전체 디베이트 실행 및 문제점 진단

`run()` 메서드를 사용하여 전체 디베이트를 한 번에 실행하고 문제점을 진단합니다.


In [ ]:
# 전체 디베이트 실행 및 문제점 진단
import traceback
from datetime import datetime

print("=" * 80)
print("전체 디베이트 실행 시작")
print("=" * 80)
print(f"시작 시간: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n")

# 새로운 DebateAgent 인스턴스 생성
debate_full = DebateAgent(rounds=ROUNDS, ticker=TICKER)

try:
    print("1️⃣ run() 메서드 실행 중...")
    print("-" * 80)
    debate_full.run()
    print("\n✅ run() 메서드 완료!")
    
    print("\n2️⃣ 최종 결과 확인...")
    print("-" * 80)
    ensemble = debate_full.get_ensemble()
    print("\n📊 최종 Ensemble 결과:")
    for key, value in ensemble.items():
        if isinstance(value, dict):
            print(f"  {key}:")
            for k, v in value.items():
                print(f"    {k}: {v}")
        else:
            print(f"  {key}: {value}")
            
except Exception as e:
    print(f"\n❌ 오류 발생: {type(e).__name__}: {e}")
    print("\n" + "=" * 80)
    print("스택 트레이스:")
    print("=" * 80)
    traceback.print_exc()
    
    # 현재 상태 확인
    print("\n" + "=" * 80)
    print("현재 상태 확인:")
    print("=" * 80)
    print(f"  opinions keys: {list(debate_full.opinions.keys())}")
    print(f"  rebuttals keys: {list(debate_full.rebuttals.keys())}")
    if debate_full.opinions:
        print(f"  마지막 라운드: {max(debate_full.opinions.keys())}")
        last_round = max(debate_full.opinions.keys())
        print(f"  마지막 라운드 의견 수: {len(debate_full.opinions[last_round])}")


## 완료

전체 토론 시스템 테스트가 완료되었습니다! 🎉

### 주요 결과 요약:
- ✅ 3개 에이전트의 초기 의견 수집 완료
- ✅ {ROUNDS}라운드의 반박 및 의견 수정 완료
- ✅ 최종 앙상블 예측 생성 완료

### 다음 단계:
- 다른 종목으로 테스트해보기
- 라운드 수를 조정하여 수렴 패턴 관찰
- Streamlit 대시보드에서 인터랙티브하게 테스트
